In [143]:
require('nn')
require('hdf5')
require('optim')

f = hdf5.open("data.hdf5", "r")

train_rnn_X = f:read('train_rnn_X'):all()
train_rnn_Y = f:read('train_rnn_Y'):all()
train_X_seqeuence =  f:read('train_X_sequence'):all()
train_Y  = f:read('train_Y'):all()
valid_reduced_X = f:read('valid_reduced_X'):all()
valid_reduced_Y = f:read('valid_reduced_Y'):all()
windows_train = f:read('windows_train'):all()
windows_valid = f:read('windows_valid'):all()
test = f:read('test'):all()
valid_kaggle_with_spaces = f:read('valid_kaggle_with_spaces'):all()
valid_kaggle_without_spaces = f:read('valid_kaggle_without_spaces'):all()
valid_answers = f:read('valid_answers'):all()
nfeatures = (f:read('nfeatures'):all())[1]
nclasses = (f:read('nclasses'):all())[1]

f:close()

window_size = windows_train:size(2)
n_actual = nclasses

In [144]:
embedded_size = 15
D_h = 100

lookup1 = nn.LookupTable(nfeatures*window_size,embedded_size)
linear1 = nn.Linear(window_size*embedded_size, D_h)
linear2 = nn.Linear(D_h, n_actual)
h = nn.Sequential()
h:add(lookup1)
--lookup1.weight:renorm(2, 2, 1)
h:add(nn.Reshape(window_size*embedded_size, True))
h:add(linear1)
h:add(nn.Tanh())
h:add(linear2)
h:add(nn.LogSoftMax())
mse = nn.ClassNLLCriterion()
--mse = nn.CrossEntropyCriterion()
w, dl_dw = h:getParameters()

In [145]:
X_set = windows_train
Y_set = train_Y

In [146]:
windows_train:size()

 599905
      2
[torch.LongStorage of size 2]



In [148]:
function feval(w_new)
   
    bsize= batch
    local idx = torch.randperm(X_set:size(1)):sub(1,bsize)

    local x = torch.Tensor(bsize, X_set:size(2))

    
    --y = y:reshape(Y_train:size(1), 1)
    
    local y_ = torch.Tensor(bsize, 1)

    for i=1,bsize do
        x[i] = X_set[idx[i]]
        y_[i] = Y_set[idx[i]]
    end
    
    y_ = y_:squeeze()
    local inputs = x
    local targets = y_
    -- reset gradients (gradients are always accumulated, to accommodate
    -- batch methods)
    dl_dw:zero()
    lookup1.weight:renorm(2, 1, 5)
    -- evaluate the loss function and its derivative with respect to x, given a mini batch
    local prediction = h:forward(inputs)
    local loss_w = mse:forward(prediction, targets)
    h:backward(inputs, mse:backward(prediction, targets))
    
    return loss_w, dl_dw
            
end

batch = 1000

-- cycle on data
for i = 1,3000 do
    -- train a mini_batch of batchSize in parallel
    _, fs = optim.adam(feval,w)

    if i % 100 == 0 then
        print('loss for iteration ' .. i  .. ' is ' .. fs[1] )
        -- print(sgd_params)
    end
end

loss for iteration 100 is 0.23900963315465	


loss for iteration 200 is 0.23902097132909	


loss for iteration 300 is 0.24802188778067	


loss for iteration 400 is 0.23186138448551	


loss for iteration 500 is 0.2417735744339	


...

loss for iteration 2600 is 0.26645682543458	


loss for iteration 2700 is 0.24566500939144	


loss for iteration 2800 is 0.24443433775536	


loss for iteration 2900 is 0.24173435534657	


loss for iteration 3000 is 0.24624991696695	


In [131]:
function perplexity(input_set, output_set)
    sum = 0
    distributions = torch.Tensor(input_set:size(1),2)
    for i = 1,1000 do
        a = (h:forward(input_set[i]))
        distribution = (nn.LogSoftMax():forward(a))
        distributions[i] = torch.exp(distribution)
        answer = distribution[output_set[i]]
        sum = sum + answer
        
    end
    return torch.exp(-sum/1000), distributions
end


In [130]:
perp, dist = perplexity(windows_valid:sub(1,windows_valid:size(1)), valid_reduced_Y)
print(perp)

1.2642763110917	


In [149]:
X_valid_nospaces = pre_pad(valid_kaggle_without_spaces, windows_train:size(2))

In [150]:
--Returns padded X_valid_nospaces for space count
function pre_pad(x_valid_nospaces, w)
    local padding = torch.Tensor(x_valid_nospaces:size(1), w-1):fill(30)
    padding = padding:type('torch.LongTensor')
    return torch.cat(padding, x_valid_nospaces, 2)
end

function NN_greedy(input_set, output_set)
    sum = 0
    distributions = torch.Tensor(input_set:size(1),2)
    local mean_error_sq = 0
    for i = 1,1000 do
        
        local spaces = 0
        local sentence = input_set:sub(i,i,1,input_set:size(2)):clone()
        
        
        for j=1,(input_set:size(2)-windows_train:size(2)) do
            
            if sentence[1][j+windows_train:size(2)] == 30 then
                break
            end
            
            local context = sentence:sub(1,1,j,j+windows_train:size(2)-1)
            local p_space = torch.exp(h:forward(context))[1][2]
            if p_space > 0.3 then
                spaces = spaces + 1
                for k=1,sentence:size(2)-j-windows_train:size(2) do
                    local idx = sentence:size(2) - k
                    sentence[1][idx+1] = sentence[1][idx]
                end
                sentence[1][j+windows_train:size(2)] = 30
            end
        end
        mean_error_sq = mean_error_sq + (spaces - output_set[i])^2
    end
    return mean_error_sq/1000
end

function NN_dp(input_set, output_set)
    sum = 0
    distributions = torch.Tensor(input_set:size(1),2)
    local mean_error_sq = 0
    for i = 1,1000 do
        
        local spaces = 0
        local sentence = input_set:sub(i,i,1,input_set:size(2)):clone()
        
        
        for j=1,(input_set:size(2)-windows_train:size(2)) do
            
            if sentence[1][j+windows_train:size(2)] == 30 then
                break
            end
            
            local context = sentence:sub(1,1,j,j+windows_train:size(2)-1)
            local p_space = torch.exp(h:forward(context))[1][2]
            if p_space > 0.3 then
                spaces = spaces + 1
                for k=1,sentence:size(2)-j-windows_train:size(2) do
                    local idx = sentence:size(2) - k
                    sentence[1][idx+1] = sentence[1][idx]
                end
                sentence[1][j+windows_train:size(2)] = 30
            end
        end
        mean_error_sq = mean_error_sq + (spaces - output_set[i])^2
    end
    return mean_error_sq/1000
end

In [151]:
mse = greedy(X_valid_nospaces, valid_answers )
print(mse)

23.685	


In [79]:
windows_valid:size()

 115716
      5
[torch.LongStorage of size 2]



In [84]:
X_valid_nospaces:size()

 3370
  420
[torch.LongStorage of size 2]



In [ ]:
function write2file(scores, fname)
    f = io.open(fname, "w")
    f:write("ID,Class1,CLass2,Class3,Class4,Class5,Class6,Class7,Class8,Class9,Class10,Class11,Class12,CLass13,Class14,Class15,Class16,Class17,Class18,Class19,Class20,Class21,Class22,Class23,CLass24,Class25,Class26,Class27,Class28,Class29,Class30,Class31,Class32,Class33,Class34,CLass35,Class36,Class37,Class38,Class39,Class40,Class41,Class42,Class43,Class44,Class45,CLass46,Class47,Class48,Class49,Class50\n")
    for i=1,scores:size(1) do
        s = tostring(i)
        for j=1, scores:size(2) do
            s = s .. "," .. tostring(scores[i][j])
        end
        f:write(s .. "\n")
    end
    f:close()
end

In [ ]:
write2file(distributions, 'predictions.txt')